<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/crawler/day_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def page_to_df(self, page):
    time = str(datetime.datetime.now().strftime("%Y%m%d%I%M%S"))
    url = "https://finance.naver.com/item/sise_time.nhn?code=" + self.code + "&thistime=" + time + "&page=" + page
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    tr = bs_obj.find_all("tr", {'onmouseover':'mouseOver(this)', 'onmouseout':"mouseOut(this)"})

    for i in range(10):
      try: 
        spans = tr[i].find_all('span')
        lines = [span.get_text() for span in spans]
        lines = list(map(lambda x: int(x.replace(',','')), lines))
        data_dict[lines[0]] = lines[1:]
      except:
        break

    df = pd.DataFrame.from_dict(data_dict).transpose()
    df.columns = ['Close','_','Sell',
                      'Buy','Volume','Change']
    df = df.drop(columns="_")
    return df

  def get_data(self, start_page:int=1, end_page:int=16):
    df = self.page_to_df(start_page)
    # 1 page = 10 days
    for i in range(start_page, end_page): 
      try: 
        if (any(self.page_to_df(i).index != self.page_to_df(i-1).index)):
          df = df.append(self.page_to_df(i))
          print('.', end = '')
          if (i%100==0):
            print(i)
        else:
          break
      except:
        df = df.append(self.page_to_df(i))
    df = df.sort_index(axis = 0) 
    for i in ['Close', 'Open', 'High', 'Low', 'Volume']:
      df[i] = df[i].astype(int)
    return df

In [47]:
from multiprocessing import Pool

stock = Stock('005930')

with Pool(500) as p:
    li = p.starmap(stock.get_data, [(x,x) for x in range(500,0,-1)])

TypeError: ignored

In [48]:
stock.page_to_df(1)

TypeError: ignored

,Close,Sell,Buy,Volume,Change
11:34,510,510,509,183694694,220412
11:33,510,511,510,183474282,107469
11:32,509,510,509,183366813,130879
11:31,510,511,510,183235934,377263
11:30,512,512,511,182858671,246113
11:29,511,512,511,182612558,223061
11:28,511,512,511,182389497,18425
11:27,512,512,511,182371072,229128
11:26,513,513,512,182141944,243878
11:25,510,510,509,181898066,143444


In [25]:
[x.get_text() for x in tr[0].find_all('span')]

['11:34',
 '510',
 '\n\t\t\t\t4\n\t\t\t\t',
 '510',
 '509',
 '183,694,694',
 '220,412']

In [58]:
url = 'https://finance.naver.com/item/sise_time.nhn?code=093230&thistime=20210527113411&page=16'
result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
bs_obj = BeautifulSoup(result.content, "html.parser")
tr = bs_obj.find_all("tr", {'onmouseover':'mouseOver(this)', 'onmouseout':"mouseOut(this)"})

for i in range(10):
  try: 
    spans = tr[i].find_all('span')
    lines = [span.get_text() for span in spans]
    lines = list(map(lambda x: int(x.replace(',','')), lines))
    data_dict[lines[0]] = lines[1:]
  except:
    break

df = pd.DataFrame.from_dict(data_dict).transpose()
df.columns = ['Close','_','Sell',
                  'Buy','Volume','Change']
df = df.drop(columns="_")

In [59]:
df

,Close,Sell,Buy,Volume,Change
11:34,510,510,509,183694694,220412
11:33,510,511,510,183474282,107469
11:32,509,510,509,183366813,130879
11:31,510,511,510,183235934,377263
11:30,512,512,511,182858671,246113
11:29,511,512,511,182612558,223061
11:28,511,512,511,182389497,18425
11:27,512,512,511,182371072,229128
11:26,513,513,512,182141944,243878
11:25,510,510,509,181898066,143444
